<a href="https://colab.research.google.com/github/jsansao/transformers_pt/blob/main/BERTimbau_Fine_tuning_CETENFolha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inferência usando transformers pré-treinados do HuggingFace 

BERTimbau

CENTENFolha - 8 partes

Batch size = 16

GPU T4x2

In [2]:
#@title Passo 1:Instalando Hugging Face Transformers
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install transformers datasets
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.9.1
# tokenizers version at notebook update --- 0.7.0

Found existing installation: tensorflow 2.6.4
Uninstalling tensorflow-2.6.4:
  Successfully uninstalled tensorflow-2.6.4
tokenizers                            0.12.1
transformers                          4.20.1


In [3]:
#@title Step 1: Loading the Dataset
#1.Load kant.txt using the Colab file manager
#2.Downloading the file from GitHub
#!curl -L  https://raw.githubusercontent.com/jsansao/corpus_ptbr/main/dump_Machado_Nobreak.txt --output "dump.txt"
!curl -L https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_aa --output "dump.txt"
#!curl -L https://raw.githubusercontent.com/jsansao/corpus_ptbr/main/Lyrics_ChicoBuarque.txt --output "kant.txt"

!awk NF < dump.txt > kant.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3724k  100 3724k    0     0  10.0M      0 --:--:-- --:--:-- --:--:-- 10.0M


In [4]:
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_aa 
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ab
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ac
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ad
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ae 
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_af
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ag
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ah

--2022-10-30 15:32:46--  https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_aa
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3813899 (3.6M) [text/plain]
Saving to: ‘cetenfolha_aa’

cetenfolha_aa       100%[===================>]   3.64M  --.-KB/s    in 0.07s   

2022-10-30 15:32:46 (49.4 MB/s) - ‘cetenfolha_aa’ saved [3813899/3813899]

--2022-10-30 15:32:47--  https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ab
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3813899 (3.6M) [text/p

In [5]:
!cat cetenfolha_aa cetenfolha_ab cetenfolha_ac cetenfolha_ad cetenfolha_ae cetenfolha_af cetenfolha_ag cetenfolha_ah>  dump.txt
!awk NF < dump.txt > kant.txt

In [6]:
#@title Passo 2:Baixando e salvando BR_BERTo
#https://huggingface.co/rdenadai/BR_BERTo

from transformers import AutoTokenizer, AutoModelForMaskedLM

#tokenizer = AutoTokenizer.from_pretrained("rdenadai/BR_BERTo")
#model = AutoModelForMaskedLM.from_pretrained("rdenadai/BR_BERTo")

tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
model = AutoModelForMaskedLM.from_pretrained('neuralmind/bert-base-portuguese-cased')

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
from datasets import load_dataset
#datasets = load_dataset("text", data_files={"train": "kant.txt", "validation": "kant_teste.txt"})
#datasets = load_dataset("text", data_files={"train": "kant.txt"}, split='train[:90%]')
ds = load_dataset("text", data_files={"train": "kant.txt"})

datasets = ds["train"].train_test_split()

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-ffd5e8c10adfedb2/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [9]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

#0:   0%|          | 0/60 [00:00<?, ?ba/s]

#1:   0%|          | 0/60 [00:00<?, ?ba/s]

#2:   0%|          | 0/60 [00:00<?, ?ba/s]

#3:   0%|          | 0/60 [00:00<?, ?ba/s]

#0:   0%|          | 0/20 [00:00<?, ?ba/s]

#1:   0%|          | 0/20 [00:00<?, ?ba/s]

#2:   0%|          | 0/20 [00:00<?, ?ba/s]

#3:   0%|          | 0/20 [00:00<?, ?ba/s]

exemplo de entrada

In [10]:
tokenized_datasets["train"][4]

{'input_ids': [101,
  530,
  7119,
  117,
  2230,
  14007,
  3554,
  11076,
  151,
  347,
  655,
  125,
  2230,
  14007,
  6356,
  5257,
  22286,
  119,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [11]:
#block_size = tokenizer.model_max_length
block_size = 32

In [12]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [13]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

#0:   0%|          | 0/60 [00:00<?, ?ba/s]

#1:   0%|          | 0/60 [00:00<?, ?ba/s]

#2:   0%|          | 0/60 [00:00<?, ?ba/s]

#3:   0%|          | 0/60 [00:00<?, ?ba/s]

#0:   0%|          | 0/20 [00:00<?, ?ba/s]

#1:   0%|          | 0/20 [00:00<?, ?ba/s]

#2:   0%|          | 0/20 [00:00<?, ?ba/s]

#3:   0%|          | 0/20 [00:00<?, ?ba/s]

In [14]:
#@title Step 11: Defining a Data Collator
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [15]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {metric.compute(predictions=predictions, references=labels)}

In [16]:
#@title Step 12: Initializing the Trainer
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./BERTimbau-CENTENFolha-8_parts",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],    
)

In [17]:
#@title Step 13: Pre-training the Model
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 173177
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 54120
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,2.451300
1000,2.361700
1500,2.325200
2000,2.326800
2500,2.300700
3000,2.282000
3500,2.263800
4000,2.264800
4500,2.232700
5000,2.252000


Saving model checkpoint to ./BERTimbau-CENTENFolha-8_parts/checkpoint-10000
Configuration saved in ./BERTimbau-CENTENFolha-8_parts/checkpoint-10000/config.json
Model weights saved in ./BERTimbau-CENTENFolha-8_parts/checkpoint-10000/pytorch_model.bin
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to ./BERTimbau-CENTENFolha-8_parts/checkpoint-20000
Configuration saved in ./BERTimbau-CENTENFolha-8_parts/checkpoint-20000/config.json
Model weights saved in ./BERTimbau-CENTENFolha-8_parts/checkpoint-20000/pytorch_model.bin
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was

TrainOutput(global_step=54120, training_loss=1.9531073046541179, metrics={'train_runtime': 12738.0585, 'train_samples_per_second': 135.952, 'train_steps_per_second': 4.249, 'total_flos': 2.848789535653248e+16, 'train_loss': 1.9531073046541179, 'epoch': 10.0})

In [18]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 57875
  Batch size = 16


Perplexity: 6.39


In [19]:
eval_results

{'eval_loss': 1.8548541069030762,
 'eval_runtime': 215.7597,
 'eval_samples_per_second': 268.238,
 'eval_steps_per_second': 16.769,
 'epoch': 10.0}

In [20]:
#@title Step 14: Saving the Final Model(+tokenizer + config) to disk
trainer.save_model("./BERTimbau-CENTENFolha-8_parts")

Saving model checkpoint to ./BERTimbau-CENTENFolha-8_parts
Configuration saved in ./BERTimbau-CENTENFolha-8_parts/config.json
Model weights saved in ./BERTimbau-CENTENFolha-8_parts/pytorch_model.bin


In [21]:
# Saving model on Wandb
import wandb
wandb.save('BERTimbau-CENTENFolha-8_parts.h5')

[]

In [22]:
#@title Step 3: Configurando o pipeline fill-mask
#@title Step 15: Language Modeling with the FillMaskPipeline
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./BERTimbau-CENTENFolha-8_parts",
    tokenizer=tokenizer
)

loading configuration file ./BERTimbau-CENTENFolha-8_parts/config.json
Model config BertConfig {
  "_name_or_path": "./BERTimbau-CENTENFolha-8_parts",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 29794
}

loading configuration file ./BERTimbau-CENTENFo

In [24]:
fill_mask("O rapaz olhou para o [MASK] ")

[{'score': 0.17389041185379028,
  'token': 8242,
  'token_str': 'céu',
  'sequence': 'O rapaz olhou para o céu'},
 {'score': 0.09678210318088531,
  'token': 528,
  'token_str': 'mar',
  'sequence': 'O rapaz olhou para o mar'},
 {'score': 0.09194570779800415,
  'token': 2979,
  'token_str': 'alto',
  'sequence': 'O rapaz olhou para o alto'},
 {'score': 0.054797518998384476,
  'token': 1341,
  'token_str': 'lado',
  'sequence': 'O rapaz olhou para o lado'},
 {'score': 0.03012910857796669,
  'token': 17508,
  'token_str': 'fotógrafo',
  'sequence': 'O rapaz olhou para o fotógrafo'}]

In [25]:
fill_mask("A moça olhou para o [MASK] ")

[{'score': 0.22396305203437805,
  'token': 8242,
  'token_str': 'céu',
  'sequence': 'A moça olhou para o céu'},
 {'score': 0.1572679579257965,
  'token': 528,
  'token_str': 'mar',
  'sequence': 'A moça olhou para o mar'},
 {'score': 0.0833396315574646,
  'token': 1341,
  'token_str': 'lado',
  'sequence': 'A moça olhou para o lado'},
 {'score': 0.06177308037877083,
  'token': 2979,
  'token_str': 'alto',
  'sequence': 'A moça olhou para o alto'},
 {'score': 0.033471353352069855,
  'token': 8105,
  'token_str': 'chão',
  'sequence': 'A moça olhou para o chão'}]

In [26]:
fill_mask("Comprou uma [MASK] na loja. ")

[{'score': 0.05365389585494995,
  'token': 6997,
  'token_str': 'máquina',
  'sequence': 'Comprou uma máquina na loja.'},
 {'score': 0.04515117406845093,
  'token': 7924,
  'token_str': 'camisa',
  'sequence': 'Comprou uma camisa na loja.'},
 {'score': 0.029280485585331917,
  'token': 9978,
  'token_str': 'placa',
  'sequence': 'Comprou uma placa na loja.'},
 {'score': 0.028624143451452255,
  'token': 11586,
  'token_str': 'fita',
  'sequence': 'Comprou uma fita na loja.'},
 {'score': 0.027298428118228912,
  'token': 4320,
  'token_str': 'peça',
  'sequence': 'Comprou uma peça na loja.'}]

In [27]:
fill_mask("A mulher não é [MASK]. ")

[{'score': 0.03964012861251831,
  'token': 3575,
  'token_str': 'diferente',
  'sequence': 'A mulher não é diferente.'},
 {'score': 0.03547123819589615,
  'token': 6648,
  'token_str': 'exceção',
  'sequence': 'A mulher não é exceção.'},
 {'score': 0.034216783940792084,
  'token': 1016,
  'token_str': 'assim',
  'sequence': 'A mulher não é assim.'},
 {'score': 0.03227553144097328,
  'token': 12222,
  'token_str': 'casada',
  'sequence': 'A mulher não é casada.'},
 {'score': 0.03098089061677456,
  'token': 13420,
  'token_str': 'perfeita',
  'sequence': 'A mulher não é perfeita.'}]

In [28]:
fill_mask("O homem não é [MASK]. ")

[{'score': 0.05757135525345802,
  'token': 13380,
  'token_str': 'perfeito',
  'sequence': 'O homem não é perfeito.'},
 {'score': 0.04005825147032738,
  'token': 2397,
  'token_str': 'homem',
  'sequence': 'O homem não é homem.'},
 {'score': 0.03814077377319336,
  'token': 2538,
  'token_str': 'Deus',
  'sequence': 'O homem não é Deus.'},
 {'score': 0.03666003793478012,
  'token': 1016,
  'token_str': 'assim',
  'sequence': 'O homem não é assim.'},
 {'score': 0.034408535808324814,
  'token': 3874,
  'token_str': 'nada',
  'sequence': 'O homem não é nada.'}]

In [29]:
fill_mask("Ele é um bom [MASK]. ")

[{'score': 0.07156675308942795,
  'token': 3620,
  'token_str': 'ator',
  'sequence': 'Ele é um bom ator.'},
 {'score': 0.06061194837093353,
  'token': 1416,
  'token_str': 'exemplo',
  'sequence': 'Ele é um bom exemplo.'},
 {'score': 0.05238033086061478,
  'token': 2357,
  'token_str': 'jogador',
  'sequence': 'Ele é um bom jogador.'},
 {'score': 0.03550665080547333,
  'token': 2481,
  'token_str': 'diretor',
  'sequence': 'Ele é um bom diretor.'},
 {'score': 0.026792114600539207,
  'token': 13254,
  'token_str': 'rapaz',
  'sequence': 'Ele é um bom rapaz.'}]

In [30]:
fill_mask("Ela é uma boa [MASK]. ")

[{'score': 0.1693785935640335,
  'token': 4067,
  'token_str': 'companhia',
  'sequence': 'Ela é uma boa companhia.'},
 {'score': 0.06881117820739746,
  'token': 8208,
  'token_str': 'opção',
  'sequence': 'Ela é uma boa opção.'},
 {'score': 0.0625816285610199,
  'token': 2760,
  'token_str': 'pessoa',
  'sequence': 'Ela é uma boa pessoa.'},
 {'score': 0.05718264728784561,
  'token': 8932,
  'token_str': 'amiga',
  'sequence': 'Ela é uma boa amiga.'},
 {'score': 0.04817567020654678,
  'token': 3557,
  'token_str': 'atriz',
  'sequence': 'Ela é uma boa atriz.'}]

In [31]:
fill_mask("Faz de conta que ainda é [MASK]. ")

[{'score': 0.13699284195899963,
  'token': 8545,
  'token_str': 'cedo',
  'sequence': 'Faz de conta que ainda é cedo.'},
 {'score': 0.09371994435787201,
  'token': 1695,
  'token_str': 'pouco',
  'sequence': 'Faz de conta que ainda é pouco.'},
 {'score': 0.07628239691257477,
  'token': 2656,
  'token_str': 'jovem',
  'sequence': 'Faz de conta que ainda é jovem.'},
 {'score': 0.06331298500299454,
  'token': 4931,
  'token_str': 'candidato',
  'sequence': 'Faz de conta que ainda é candidato.'},
 {'score': 0.058164678514003754,
  'token': 1373,
  'token_str': 'tarde',
  'sequence': 'Faz de conta que ainda é tarde.'}]

In [32]:
fill_mask("Depois da tempestade vem a [MASK]. ")

[{'score': 0.2662242650985718,
  'token': 8742,
  'token_str': 'seca',
  'sequence': 'Depois da tempestade vem a seca.'},
 {'score': 0.05305907130241394,
  'token': 1386,
  'token_str': 'morte',
  'sequence': 'Depois da tempestade vem a morte.'},
 {'score': 0.052535828202962875,
  'token': 12299,
  'token_str': 'neve',
  'sequence': 'Depois da tempestade vem a neve.'},
 {'score': 0.03662586212158203,
  'token': 10186,
  'token_str': 'primavera',
  'sequence': 'Depois da tempestade vem a primavera.'},
 {'score': 0.03568654507398605,
  'token': 9856,
  'token_str': 'chuva',
  'sequence': 'Depois da tempestade vem a chuva.'}]

In [33]:
fill_mask("Mais vale um pássaro na mão do que [MASK] voando. ")

[{'score': 0.35812652111053467,
  'token': 222,
  'token_str': 'um',
  'sequence': 'Mais vale um pássaro na mão do que um voando.'},
 {'score': 0.09153059870004654,
  'token': 1342,
  'token_str': 'outro',
  'sequence': 'Mais vale um pássaro na mão do que outro voando.'},
 {'score': 0.07664812356233597,
  'token': 14934,
  'token_str': 'voar',
  'sequence': 'Mais vale um pássaro na mão do que voar voando.'},
 {'score': 0.048725761473178864,
  'token': 6384,
  'token_str': 'alguém',
  'sequence': 'Mais vale um pássaro na mão do que alguém voando.'},
 {'score': 0.04480363428592682,
  'token': 5197,
  'token_str': 'sair',
  'sequence': 'Mais vale um pássaro na mão do que sair voando.'}]

In [34]:
fill_mask("Não tinha [MASK], mas almoçou mesmo assim. ")

[{'score': 0.08342432975769043,
  'token': 3495,
  'token_str': 'dinheiro',
  'sequence': 'Não tinha dinheiro, mas almoçou mesmo assim.'},
 {'score': 0.051540203392505646,
  'token': 11572,
  'token_str': 'telefone',
  'sequence': 'Não tinha telefone, mas almoçou mesmo assim.'},
 {'score': 0.035101037472486496,
  'token': 12319,
  'token_str': 'chegado',
  'sequence': 'Não tinha chegado, mas almoçou mesmo assim.'},
 {'score': 0.033241044729948044,
  'token': 13746,
  'token_str': 'namorado',
  'sequence': 'Não tinha namorado, mas almoçou mesmo assim.'},
 {'score': 0.027934936806559563,
  'token': 14890,
  'token_str': 'jantar',
  'sequence': 'Não tinha jantar, mas almoçou mesmo assim.'}]

In [35]:
fill_mask("Bebeu um copo d'água, pois tinha [MASK]. ")

[{'score': 0.22507280111312866,
  'token': 11062,
  'token_str': 'fome',
  'sequence': "Bebeu um copo d'água, pois tinha fome."},
 {'score': 0.15004238486289978,
  'token': 14594,
  'token_str': 'febre',
  'sequence': "Bebeu um copo d'água, pois tinha febre."},
 {'score': 0.12500622868537903,
  'token': 7672,
  'token_str': 'medo',
  'sequence': "Bebeu um copo d'água, pois tinha medo."},
 {'score': 0.043515708297491074,
  'token': 2496,
  'token_str': 'sede',
  'sequence': "Bebeu um copo d'água, pois tinha sede."},
 {'score': 0.0227068904787302,
  'token': 17609,
  'token_str': 'acabado',
  'sequence': "Bebeu um copo d'água, pois tinha acabado."}]

In [36]:
fill_mask("Sem saber, ele descobriu um [MASK]. ")

[{'score': 0.1839371919631958,
  'token': 11021,
  'token_str': 'segredo',
  'sequence': 'Sem saber, ele descobriu um segredo.'},
 {'score': 0.13480046391487122,
  'token': 3420,
  'token_str': 'caminho',
  'sequence': 'Sem saber, ele descobriu um caminho.'},
 {'score': 0.03356393426656723,
  'token': 16269,
  'token_str': 'mistério',
  'sequence': 'Sem saber, ele descobriu um mistério.'},
 {'score': 0.018778884783387184,
  'token': 3350,
  'token_str': 'problema',
  'sequence': 'Sem saber, ele descobriu um problema.'},
 {'score': 0.016541659832000732,
  'token': 3204,
  'token_str': 'plano',
  'sequence': 'Sem saber, ele descobriu um plano.'}]

In [37]:
fill_mask("Pedro fez fortuna vendendo [MASK]. ")

[{'score': 0.07013530284166336,
  'token': 2978,
  'token_str': 'livros',
  'sequence': 'Pedro fez fortuna vendendo livros.'},
 {'score': 0.06996648758649826,
  'token': 3191,
  'token_str': 'armas',
  'sequence': 'Pedro fez fortuna vendendo armas.'},
 {'score': 0.05868770554661751,
  'token': 9701,
  'token_str': 'cavalos',
  'sequence': 'Pedro fez fortuna vendendo cavalos.'},
 {'score': 0.05046388506889343,
  'token': 2987,
  'token_str': 'ouro',
  'sequence': 'Pedro fez fortuna vendendo ouro.'},
 {'score': 0.03814943507313728,
  'token': 5976,
  'token_str': 'escravos',
  'sequence': 'Pedro fez fortuna vendendo escravos.'}]

In [39]:
fill_mask("Sem medo de ser [MASK]. ")

[{'score': 0.11592359840869904,
  'token': 5139,
  'token_str': 'preso',
  'sequence': 'Sem medo de ser preso.'},
 {'score': 0.07185672223567963,
  'token': 8540,
  'token_str': 'feliz',
  'sequence': 'Sem medo de ser feliz.'},
 {'score': 0.04520270973443985,
  'token': 13205,
  'token_str': 'demitido',
  'sequence': 'Sem medo de ser demitido.'},
 {'score': 0.03822215273976326,
  'token': 14133,
  'token_str': 'expulso',
  'sequence': 'Sem medo de ser expulso.'},
 {'score': 0.03504673019051552,
  'token': 15854,
  'token_str': 'pega',
  'sequence': 'Sem medo de ser pega.'}]

In [40]:
wandb.finish()

eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,█▇▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
